In [ ]:
import numpy as np
import scipy.linalg as spla
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

from pymor.algorithms.gram_schmidt import gram_schmidt
from pymor.algorithms.svd_va import mos, qr_svd
from pymor.core.exceptions import AccuracyError
from pymor.vectorarrays.numpy import NumpyVectorSpace

# Setup

In [ ]:
n = 1000
m = 50
random = np.random.RandomState(0)
U = spla.qr(random.standard_normal((n, m)), mode='economic')[0]
s = np.logspace(-20, 1, m)[::-1]
Vh = spla.qr(random.standard_normal((m, m)), mode='economic')[0]
A = U * s @ Vh.T
Ava = NumpyVectorSpace(n).from_numpy(A.T)

In [ ]:
U_svd, s_svd, Vh_svd = spla.svd(A, lapack_driver='gesvd')

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(s, '.-', label='exact')
ax.semilogy(s_svd, '.-', label='gesvd')
ax.set_title('Singular values')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(np.abs(s_svd - s) / np.sqrt(s_svd * s), '.-')
ax.set_title('Relative error of gesvd singular values')
plt.show()

In [ ]:
def sin_max_angle(V, W):
    """Sine of the maximum princial angle between two subspaces given by orthonormal bases."""
    return spla.norm(V - W @ W.T @ V, ord=2)

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy([sin_max_angle(U_svd[:, :r], U[:, :r]) for r in range(1, m + 1)], '.-')
ax.set_title('Sine of maximum principal angle between dominant left subspaces')
plt.show()

# Method of snapshots

## Without additional orthonormalization

In [ ]:
try:
    mos(Ava)
except AccuracyError as e:
    print(e)

In [ ]:
U_mos_wo, _, _ = mos(Ava, check=False)

In [ ]:
Q_mos_wo, R_mos_wo = gram_schmidt(U_mos_wo, return_R=True, atol=0, rtol=0)

In [ ]:
spla.svdvals(R_mos_wo)

## With additional orthonormalization

In [ ]:
U_mos, s_mos, Vh_mos = mos(Ava, rtol=0, check=False)
U_mos = U_mos.to_numpy().T

In [ ]:
len(s_mos)

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(s, '.-', label='exact')
ax.semilogy(s_svd, '.-', label='gesvd')
ax.semilogy(s_mos, '.-', label='mos')
ax.set_title('Singular values')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(np.abs(s_svd - s), '.-', label='gesvd - exact')
ax.semilogy(np.abs(s_mos - s[:len(s_mos)]), '.-', label='mos - exact')
ax.set_title('Absolute distances between singular values')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(np.abs(s_svd - s) / np.sqrt(s_svd * s), '.-', label='gesvd - exact')
ax.semilogy(np.abs(s_mos - s[:len(s_mos)]) / np.sqrt(s_mos * s[:len(s_mos)]), '.-', label='mos - exact')
ax.set_title('Relative distances between singular values')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy([sin_max_angle(U_svd[:, :r], U[:, :r]) for r in range(1, m + 1)], '.-', label='gesvd - exact')
ax.semilogy([sin_max_angle(U_mos[:, :r], U[:, :r]) for r in range(1, len(s_mos))], '.-', label='mos - exact')
ax.set_title('Sine of maximum principal angle between dominant left subspaces')
ax.legend()
plt.show()

# QR + SVD

In [ ]:
U_qr, s_qr, Vh_qr = qr_svd(Ava, rtol=0)
U_qr = U_qr.to_numpy().T

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(s, '.-', label='exact')
ax.semilogy(s_svd, '.-', label='gesvd')
ax.semilogy(s_qr, '.--', label='qr_svd')
ax.set_title('Singular values')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(np.abs(s_svd - s), '.-', label='gesvd - exact')
ax.semilogy(np.abs(s_qr - s), '.-', label='qr_svd - exact')
ax.set_title('Absolute distances between singular values')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(np.abs(s_svd - s) / np.sqrt(s_svd * s), '.-', label='gesvd - exact')
ax.semilogy(np.abs(s_qr - s) / np.sqrt(s_qr * s), '.-', label='qr_svd - exact')
ax.set_title('Relative distances between singular values')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy([sin_max_angle(U_svd[:, :r], U[:, :r]) for r in range(1, m + 1)], '.-', label='gesvd - exact')
ax.semilogy([sin_max_angle(U_qr[:, :r], U[:, :r]) for r in range(1, m + 1)], '.-', label='qr_svd - exact')
ax.set_title('Sine of maximum principal angle between dominant left subspaces')
ax.legend()
plt.show()